In [ ]:
import sys; sys.path.append('..')
from bending_validation import *
import elastic_rods, sparse_matrices, pickle, scipy, linkage_vis, numpy as np, time

In [ ]:
L, a, n = 1, 0.1, 51
perturb = False
r, fixedVars = bendingTestRod(L, a, n)
pts, thetas = initialConfiguration(L, a, n, perturb)
thetas = np.ones(n - 1) * (np.pi / 2)
r.setDeformedConfiguration(pts, thetas)

view = linkage_vis.LinkageViewer(r, width=1024, height=600)
view.setCameraParams(((0.8471149305506829, 2.303483674764291, 0.4579086561101697),
 (-0.05274977528957473, 0.1226800644593855, 0.9910434213449677),
 (-0.41643357695558714, 0.21249216220933723, -0.36668979830898635)))
view.show()

In [ ]:
view.averagedMaterialFrames = not view.averagedMaterialFrames

In [ ]:
# elastic_rods.compute_equilibrium_knitro(r, verbose=True, fixedVars=fixedVars, niter=10)
with suppress_stdout(): elastic_rods.compute_equilibrium(r, fixedVars=fixedVars)
view.update()

In [ ]:
view.update(scalarField=r.maxBendingStresses())

In [ ]:
from compute_vibrational_modes import compute_vibrational_modes, MassMatrixType
import mode_viewer

lambdas, modes = compute_vibrational_modes(r, fixedVars)
mview = mode_viewer.ModeViewer(r, modes, lambdas)
mview.show()

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
iplot = interactive(runTestVisualization,
                    L=fixed(1),
                    a=widgets.FloatSlider(min=0.01, max=1, value=0.5, step=0.01),
                    numVertices=widgets.IntSlider(min=3,max=100,step=2,value=41),
                    gradTol=fixed(1e-12))
iplot.children[-1].layout.height = '450px'
display(iplot)

In [ ]:
t = time.time()
totalIters = convergenceTest(0.6, 1000, 1e-11)
print(time.time() - t, totalIters)

# Gravity Demo (Cantilever)
Clamp a 2m long 12x8mm GFRP rod at one end and expose to gravity.

In [ ]:
r = elastic_rods.ElasticRod(np.linspace([0, 0, 0], [2000, 0, 0], 50))
# Common GFRP Young's modulus: 20GPa = 20000MPa
mat = elastic_rods.RodMaterial('Rectangle', 20000.0, 0.3, [12, 8])
r.setMaterial(mat)

In [ ]:
# Note: if we make the rod cross-section anisotropic
# and then rotate it around the centerline by some angle, the gravitational
# and force ends up pulling the rod out of x-z plane and applying a twist due to the anisotropic bending energy.
# This doesn't happen for circular cross-sections or if the cross-section is oriented perfectly horizontal or vertical.
d = r.getDoFs()
d[r.thetaOffset():] = np.pi / 2
r.setDoFs(d)

In [ ]:
gview = linkage_vis.LinkageViewer(r)
gview.show()

In [ ]:
# Use a common density for GFRP
# 2000 kg/m^3 = 2e-6 kg/mm^3
f = r.gravityForce(2e-6)
with suppress_stdout(): elastic_rods.compute_equilibrium(r, f, fixedVars=[0, 1, 2, 3, 4, 5, r.thetaOffset()])

gview.ghostColor='lightgray'
gview.update(transparent=True)

# The reaction forces applied by the clamp conditions drown out the gravitational forces if we try to visualize them at the same time....
elasticForces = -r.gradient()[0:r.thetaOffset()].reshape((-1, 3))
visualizationForces = -elasticForces.copy()
visualizationForces[0:2, :] = 0
cview = linkage_vis.CenterlineViewer(r, vectorField=visualizationForces, superView=gview)